In [47]:
from openai import OpenAI
import json

In [14]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/U OF T/MIE/ECE1786/Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/U OF T/MIE/ECE1786/Project


## Recipe Generation

In [49]:
# evaluation criteria; sample recipes; user input
# quantitative comparison result

In [53]:
client = OpenAI(api_key="")
file_path = "/content/drive/MyDrive/U OF T/MIE/ECE1786/Project/test_processed_recipes.json"

In [50]:
def get_sys_prompt(file_path):
  sample_recipe = load_file_content(file_path)
  recipe_examples = []
  for recipe in sample_recipe:
    # Extract title, ingredients, and instructions safely
    title = recipe.get("recipe_title", "Untitled Recipe")
    ingredients = recipe.get("ingredients", [])
    instructions = recipe.get("processed_output", {}).get("step_by_step_instructions", [])

    # Format the extracted recipe as a string
    formatted_recipe = (
        f"Title: {title}\n"
        f"Ingredients: {', '.join(ingredients)}\n"
        f"Instructions: {' '.join(instructions)}\n"
    )
    recipe_examples.append(formatted_recipe)

  sample_recipes_text = "\n\n".join(recipe_examples)

  sys_prompt = f'''
    You are a helpful assistant that can generate a healthy recipe based on a sample recipe and evaluate its health score.

    Here's the evaluation criteria:
    You only need to consider 7 key macronutrients and their ranges to assess a recipe's healthiness:
    Proteins: 10%-15% of total energy
    Carbohydrates: 55%-75% of total energy
    Sugars: less than 10% of total energy
    Sodium: less than 5 grams
    Fats: 15%-30% of total energy
    Saturated Fats: less than 10% of total energy
    Fibers: more than 25 grams

    The instructions of the evaluation metric:
    1. For each nutrient, evaluate whether its amount in the recipe falls within the given range.
    2. Assign 1 point for each nutrient that falls within the range, and 0 points for each nutrient that falls out of the range.
    3. Add all the points to get the health score. The range of the health score is from 0 to 7.

    Here's a sample recipe:
    {sample_recipes_text}

    The sample evaluation result and health score for the first recipe:
    Summary of Points:
    Proteins: 0 points
    Carbohydrates: 1 point (potatoes provide substantial carbohydrates)
    Sugars: 1 point (natural sugars in potatoes and cream are likely to be less than 10% of total energy)
    Sodium: 1 point (assuming moderate salt use, likely to stay under 5 grams)
    Fats: 1 point (butter, olive oil, cream are high in fats, possibly within the 15%-30% range)
    Saturated Fats: 0 points (butter, cream, and sour cream likely push this over 10%)
    Fibers: 0 points (likely less than 25 grams, as potatoes are not high in fibers)
    Total Health Score: 4

    Now, your job is to generate a recipe that has the highest health score based on the user input of the ingredients and cooking tools.
    Note, you must only use the ingredients that the user have.
    Provide the instructions line by line and number them.
    Then, evaluate the recipe you generated based on the evaluation criteria.
    The format of the recipe and evaluation result should be the same as the sample recipe and sample evaluation result.
    Lastly, provide some suggestions to the recipe based on the evaluation result.
  '''
  return sys_prompt

In [20]:
def load_file_content(file_path):
    """Loads and returns the content of the file as a string."""
    try:
        with open(file_path, "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return "File not found. Please check the file path."

In [54]:
def get_recipe(client, ingredients, tools, temp, topp, file_path):

    user_prompt = (
        f"I have the following ingredients: {', '.join(ingredients)}.\n"
        f"I also have these cooking tools: {', '.join(tools)}.\n"
    )
    sys_prompt = get_sys_prompt(file_path)

    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=temp,
            top_p=topp
        )

        return completion.choices[0].message.content.strip()
    except Exception as e:
        return f"An error occurred: {e}"

In [55]:
# main
print("Welcome to RecipePrep!")

ingredients_input = input("Enter your ingredients, separated by commas: ").strip().split(",")
avail_ingredients = [ingredient.strip() for ingredient in ingredients_input]

tools_input = input("Enter your cooking tools, separated by commas: ").strip().split(",")
avail_tools = [tool.strip() for tool in tools_input]

# Generate recipe using the inputs
print("\nGenerating your recipe...\n")
recipe = get_recipe(client, avail_ingredients, avail_tools, 1, 1, file_path)
print("Here's your recipe:\n")
print(recipe)

Welcome to RecipePrep!
Enter your ingredients, separated by commas: chicken, onion, garlic, salt, sugar
Enter your cooking tools, separated by commas: pots, oven

Generating your recipe...

Here's your recipe:

Title: Garlic Chicken with Onions

Ingredients: 
- 2 chicken breasts
- 2 onions, sliced
- 4 cloves of garlic, minced
- 1 teaspoon salt
- 1 teaspoon sugar

Instructions:
1. Preheat the oven to 375°F (190°C).
2. In a large pot, place the sliced onions and minced garlic evenly across the bottom.
3. Place the chicken breasts on top of the onions and garlic.
4. Season the chicken with salt.
5. Sprinkle sugar evenly over the chicken and onions.
6. Cover the pot and place it in the preheated oven.
7. Bake for about 45 minutes, or until the chicken is cooked through and the onions are caramelized.
8. Once cooked, remove the pot from the oven and let it rest for 5 minutes before serving.

Evaluation Result:

Summary of Points:
- Proteins: 1 point (chicken is a good source of protein)
- C